<center><h1>Phishing Classificaiton</h1></center>

In [101]:
import pandas as pd
import seaborn as sns
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from scipy import stats

In [102]:
df = pd.read_csv('parsed_url_features.csv')
# drop non numeric columns
for col in df.columns:
    if df[col].dtype == 'object':
        df = df.drop(col, axis=1)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(inplace=True)
df.head(10)

,status,n_period,n_hyphens,n_underscore,n_slash,n_questionmrk,n_equals,n_at,n_and,n_exclamation,url_length,domain_name_correct
0,0.0,2,0,0,3,0,0,0,0,0,25,1
1,0.0,2,1,0,3,0,0,0,0,0,35,1
2,0.0,2,1,0,0,0,0,0,0,0,35,1
3,0.0,1,0,0,0,0,0,0,0,0,9,0
4,0.0,3,0,0,10,1,0,0,0,0,74,0
5,0.0,3,2,0,0,0,0,0,0,0,36,0
6,0.0,3,0,0,7,0,0,0,0,0,54,0
7,0.0,2,1,1,7,0,0,0,0,0,74,1
8,0.0,5,2,0,3,0,0,0,0,0,50,1
9,0.0,4,0,0,0,0,0,0,0,0,21,1


In [103]:
# determimg the number of outliers
clf = IsolationForest(contamination=0.005)
X = df.iloc[:, 1:].dropna()
y_pred = clf.fit_predict(X)
outliers = np.where(y_pred==-1)
outliers = outliers[0]
print(outliers)
print('Number of outliers:', len(outliers))
print(df.shape)

[  347   447   571   610   675   703   715   724  1012  1066  1176  1239
  1245  1280  1468  1643  1734  1940  1981  2063  2095  2128  2208  2210
  2379  2595  2636  2678  2785  3040  3047  3081  3280  3445  3470  3531
  3589  3730  3779  3817  3863  3917  4039  4175  4682  4773  4784  5012
  5092  5097  5164  5318  5360  5416  5569  5751  6009  6070  6345  6545
  6907  6924  6956  6994  7541  7723  7885  7900  7996  8014  8044  8046
  8163  8362  8480  8538  8567  8669  8795  8823  8856  8860  9029  9115
  9513  9516  9843  9899 10172 10196 10406 10536 10560 10759 10766 10881
 10942 11054 11247 11288 11595 11732 11835 12054 12061 12322 12378 12644
 12669 12727 12733 12922 13167 13304 13356 13587 13616 13682 13790 13892
 14042 14135 14202 14270 14678 14786 15027 15133 15233 15264 15318 15521
 15651 15785 15799 15815 15840 15990 16000 16035 16310 16328 16336 16381
 16394 16441 16481 16489 16589 16608 16680 16837 17009 17060 17298 17442
 17452 17545 17571 17839 17887 18054 18186 18189 18

In [104]:
outliers_exist = df.index.isin(outliers)
df_scaled = df[~outliers_exist]
print('Number of rows after dropping outliers:', df_scaled.shape[0])

Number of rows after dropping outliers: 98095


<h2>Splitting and Preprocessing the Data<h2>

In [105]:
y = df['status']
X = df.drop(columns=['status'])

# use kfold cross validation
kf = KFold(n_splits=5, shuffle=True)

In [106]:
scaler = StandardScaler()
fold_no = 1
for train, test, in kf.split(X, y):
    X = scaler.fit_transform(X)
    X_train, X_test = X[train], X[test]
    y_train, y_test = y.iloc[train], y.iloc[test]
    
    # train the model
    model = LocalOutlierFactor(n_neighbors=20, contamination=0.005, novelty=True)
    y_pred = model.fit(X_train, y_train)
    
    # evaluate the model
    score = model.score_samples(X_test)
    # find the mean of the score
    score = score.mean()
    print(f'Score for fold {fold_no}: {score*100}%')
    fold_no += 1

Score for fold 1: -942102547.6385761%
Score for fold 2: -388821852.20946443%
Score for fold 3: -643321757.1489563%
Score for fold 4: -538967887.3481951%
Score for fold 5: -737250944.5477134%


In [107]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

logistic_model = LogisticRegression()

logistic_scores = cross_val_score(logistic_model, X, y, cv=kfold)

random_forest_model = RandomForestClassifier()

random_forest_scores = cross_val_score(random_forest_model, X, y, cv=kfold)

print(f'Logistic Regression scores: {logistic_scores}')
print(f'Random Forest scores: {random_forest_scores}')

Logistic Regression scores: [0.64630992 0.65112858 0.64345135 0.6493355  0.64705286]
Random Forest scores: [0.8333756  0.83028151 0.83062798 0.8296642  0.83291062]


In [108]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
logistic_accuracy = []
logistic_sensitivity = []
logistic_specificity = []
logistic_auc = []

random_forest_accuracy = []
random_forest_sensitivity = []
random_forest_specificity = []
random_forest_auc = []

fold_no = 1
for train, test in kf.split(X, y):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y.iloc[train], y.iloc[test]
    
    logistic_model.fit(X_train, y_train)
    logistic_y_pred = logistic_model.predict(X_test)
    
    logistic_accuracy.append(accuracy_score(y_test, logistic_y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, logistic_y_pred).ravel()
    logistic_sensitivity.append(tp / (tp + fn))
    logistic_specificity.append(tn / (tn + fp))
    logistic_auc.append(roc_auc_score(y_test, logistic_y_pred))
    
    random_forest_model.fit(X_train, y_train)
    random_forest_y_pred = random_forest_model.predict(X_test)
    
    random_forest_accuracy.append(accuracy_score(y_test, random_forest_y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, random_forest_y_pred).ravel()
    random_forest_sensitivity.append(tp / (tp + fn))
    random_forest_specificity.append(tn / (tn + fp))
    random_forest_auc.append(roc_auc_score(y_test, random_forest_y_pred))
    
    print(f"Metrics for fold {fold_no}:")
    print(f"Logistic Regression - Accuracy: {logistic_accuracy[-1]}, Sensitivity: {logistic_sensitivity[-1]}, Specificity: {logistic_specificity[-1]}, AUC: {logistic_auc[-1]}")
    print(f"Random Forest - Accuracy: {random_forest_accuracy[-1]}, Sensitivity: {random_forest_sensitivity[-1]}, Specificity: {random_forest_specificity[-1]}, AUC: {random_forest_auc[-1]}")
    print()
    
    fold_no += 1

print("Average Metrics:")
print(f"Logistic Regression - Accuracy: {sum(logistic_accuracy) / len(logistic_accuracy)}, Sensitivity: {sum(logistic_sensitivity) / len(logistic_sensitivity)}, Specificity: {sum(logistic_specificity) / len(logistic_specificity)}, AUC: {sum(logistic_auc) / len(logistic_auc)}")
print(f"Random Forest - Accuracy: {sum(random_forest_accuracy) / len(random_forest_accuracy)}, Sensitivity: {sum(random_forest_sensitivity) / len(random_forest_sensitivity)}, Specificity: {sum(random_forest_specificity) / len(random_forest_specificity)}, AUC: {sum(random_forest_auc) / len(random_forest_auc)}")

Metrics for fold 1:
Logistic Regression - Accuracy: 0.6499112351001776, Sensitivity: 0.6590125160145855, Specificity: 0.6402591973244147, AUC: 0.6496358566695002
Random Forest - Accuracy: 0.8295206695409587, Sensitivity: 0.8471469399822608, Specificity: 0.8108277591973244, AUC: 0.8289873495897926

Metrics for fold 2:
Logistic Regression - Accuracy: 0.6453969059092062, Sensitivity: 0.6552717877372553, Specificity: 0.6351015333609614, AUC: 0.6451866605491084
Random Forest - Accuracy: 0.8329190971341618, Sensitivity: 0.8569015204213455, Specificity: 0.807915457936179, AUC: 0.8324084891787622

Metrics for fold 3:
Logistic Regression - Accuracy: 0.646799228974333, Sensitivity: 0.6571657165716571, Specificity: 0.6361296963458569, AUC: 0.646647706458757
Random Forest - Accuracy: 0.8284975144567313, Sensitivity: 0.8503850385038504, Specificity: 0.8059701492537313, AUC: 0.8281775938787909

Metrics for fold 4:
Logistic Regression - Accuracy: 0.6499949274627168, Sensitivity: 0.6547314578005116, S

In [109]:
metrics = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest'],
    'Accuracy': [sum(logistic_accuracy) / len(logistic_accuracy), sum(random_forest_accuracy) / len(random_forest_accuracy)],
    'Sensitivity': [sum(logistic_sensitivity) / len(logistic_sensitivity), sum(random_forest_sensitivity) / len(random_forest_sensitivity)],
    'Specificity': [sum(logistic_specificity) / len(logistic_specificity), sum(random_forest_specificity) / len(random_forest_specificity)],
    'AUC': [sum(logistic_auc) / len(logistic_auc), sum(random_forest_auc) / len(random_forest_auc)]
})
metrics.head()

,Model,Accuracy,Sensitivity,Specificity,AUC
0,Logistic Regression,0.647222,0.655551,0.638489,0.647020
1,Random Forest,0.831068,0.853044,0.807973,0.830509


In [110]:
t_statistic, p_value = stats.ttest_rel(logistic_accuracy, random_forest_accuracy)
print(f'Accuracy: t-statistic = {t_statistic}, p-value = {p_value}')

t_statistic, p_value = stats.ttest_rel(logistic_sensitivity, random_forest_sensitivity)
print(f'Sensitivity: t-statistic = {t_statistic}, p-value = {p_value}')

t_statistic, p_value = stats.ttest_rel(logistic_auc, random_forest_auc)
print(f'AUC: t-statistic = {t_statistic}, p-value = {p_value}')

Accuracy: t-statistic = -86.09608185526017, p-value = 1.0910061690255404e-07
Sensitivity: t-statistic = -65.25858928952692, p-value = 3.3030943093591973e-07
AUC: t-statistic = -85.94937311415222, p-value = 1.0984709322043184e-07


In [111]:
# figure out what the best features are for each model, then print out the top 3 features
logistic_model.fit(X, y)
random_forest_model.fit(X, y)

rf_importances = random_forest_model.feature_importances_

lr_importances = np.abs(logistic_model.coef_)
feature_names = df.columns



print("Random Forest:")
for i in np.argsort(rf_importances)[::-1][:3]:
    print(f"{feature_names[i]}: {rf_importances[i]}")


print("Logistic Regression:")
for i in np.argsort(lr_importances)[::-1][:3]:
    print(f"{feature_names[i]}: {lr_importances[0][i]}")
    


Random Forest:
n_underscore: 0.3706620348989005
n_exclamation: 0.2779316787644186
status: 0.13786020383007686
Logistic Regression:
Index(['n_and', 'n_slash', 'n_underscore', 'url_length', 'n_equals',
       'n_exclamation', 'n_at', 'n_questionmrk', 'status', 'n_period',
       'n_hyphens'],
      dtype='object'): [0.03054741 0.04013388 0.08637364 0.17258667 0.24022962 0.40019982
 0.40975636 0.42976707 0.59861649 0.79453022 0.84656169]
